In [ ]:
#카카오API - 서울시 지하철역 

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm

def get_subway_stations_in_seoul(api_key, gu):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    all_stations = []
    page = 1

    while True:
        params = {
            "query": f"{gu} 지하철역",
            "page": page
        }

        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        if "documents" in data:
            subway_stations = data["documents"]
            all_stations.extend(subway_stations)
        else:
            print(f"{gu} 지하철역 정보를 가져올 수 없습니다.")
            return None

        # 다음 페이지가 있는지 확인
        meta = data.get("meta", {})
        is_end = meta.get("is_end", True)
        if is_end:
            break

        # 다음 페이지 요청을 위해 페이지 번호 증가
        page += 1

    return all_stations

# 서울특별시의 구 리스트
seoul_gu_list = ["강남구", "강동구", "강북구", "강서구", "관악구", "광진구", "구로구", "금천구", 
                 "노원구", "도봉구", "동대문구", "동작구", "마포구", "서대문구", "서초구", "성동구",
                 "성북구", "송파구", "양천구", "영등포구", "용산구", "은평구", "종로구", "중구", "중랑구"]

# 모든 구의 지하철역 정보 가져오기
all_seoul_subway_stations = []
total_stations = 0

for gu in seoul_gu_list:
    stations = get_subway_stations_in_seoul(API_KEY, gu)
    if stations:
        all_seoul_subway_stations.extend(stations)
        total_stations += len(stations)
    else:
        print("오류 발생: 지하철역 정보를 가져오지 못했습니다.")

# '서울'로 시작하는 주소만 필터링
seoul_subway_stations = [station for station in all_seoul_subway_stations if station['address_name'].startswith('서울')]

# 결과를 데이터프레임으로 변환
if seoul_subway_stations:
    df = pd.DataFrame(seoul_subway_stations)
    print("서울에 있는 모든 지하철역 정보:")
    print(df)
    print(f"\n총 {total_stations}개의 지하철역이 서울특별시에 있습니다.")
else:
    print("서울 지하철역 정보를 가져오지 못했습니다.")


서울에 있는 모든 지하철역 정보:
          address_name category_group_code category_group_name  \
0       서울 강남구 역삼동 858                 SW8                지하철역   
1    서울 서초구 서초동 1366-9                 SW8                지하철역   
2    서울 강남구 삼성동 172-66                 SW8                지하철역   
3       서울 강남구 역삼동 800                 SW8                지하철역   
4    서울 강남구 삼성동 172-66                 SW8                지하철역   
..                 ...                 ...                 ...   
437     서울 중랑구 망우동 320                 SW8                지하철역   
438      서울 중랑구 신내동 19                                           
439     서울 중랑구 상봉동 172                 SW8                지하철역   
440       서울 중랑구 묵동 30                                           
441    서울 중랑구 중화동 73-7                 SW8                지하철역   

               category_name distance          id         phone  \
0    교통,수송 > 지하철,전철 > 수도권2호선             21160803  02-6110-2221   
1    교통,수송 > 지하철,전철 > 수도권3호선             21160815  02-

In [1]:
#좌표변환

In [12]:
import pandas as pd
import math
import requests
from tqdm import tqdm

# 카카오맵 API 키
API_KEY = "b3d07b3bc9d36308dfc4b023582e291d"

# 검색 반경 (미터)
search_radius = 5000

def calculate_distance(lat1, lon1, lat2, lon2):
    # 지구 반지름 (단위: km)
    R = 6371.0

    # 라디안으로 변환
    lat1_rad = math.radians(float(lat1))
    lon1_rad = math.radians(float(lon1))
    lat2_rad = math.radians(float(lat2))
    lon2_rad = math.radians(float(lon2))

    # 위도 및 경도 간의 차이 계산
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # Haversine 공식을 사용하여 거리 계산
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c * 1000 #m단위로 변경

    return distance

def get_nearest_subway_station(latitude, longitude, radius, api_key):
    url = "https://dapi.kakao.com/v2/local/search/category.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {
        "category_group_code": "SW8",  # SW8는 지하철역 카테고리 코드
        "y": latitude,
        "x": longitude,
        "radius": radius,
        "sort": "distance",
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if "meta" in data and data["meta"]["total_count"] > 0:
        nearest_station = data["documents"][0]

        # 선택된 지하철역 정보 추출
        station_name = nearest_station["place_name"]
        station_latitude = round(float(nearest_station["y"]), 6)
        station_longitude = round(float(nearest_station["x"]), 6)
        station_distance = calculate_distance(latitude, longitude, station_latitude, station_longitude)

        return {
            "name": station_name,
            "latitude": station_latitude,
            "longitude": station_longitude,
            "distance": station_distance
        }
    else:
        return None

# 엑셀 파일 경로
excel_file_path = '거리계산(중복제거)/DB_변수추가/분석용데이터_전처리완/연립다세대_매매_이상치처리.csv'

# 결과를 저장할 리스트 초기화
result_list = []

# 엑셀 파일에서 데이터 프레임으로 읽어오기
apart = pd.read_csv(excel_file_path, encoding='euc-kr')[2515:2520]

# 각 단지명에 대해 가장 가까운 지하철역 정보 찾기
for index, row in tqdm(apart.iterrows(), total=apart.shape[0]):
    apart_name = row['단지명']
    latitude = row['단지 위도']
    longitude = row['단지 경도']
    subway_station = row['지하철역']  # 지하철역 변수 추가
    distance_subway = row['거리(m)']

    # 지하철역이 0인 경우에만 거리 계산 수행
    if subway_station == 0 and not row.isna().any():
        # 주어진 위치 주변의 가장 가까운 지하철역 찾기
        nearest_subway_station = get_nearest_subway_station(latitude, longitude, search_radius, API_KEY)
        if nearest_subway_station:
            # 결과를 딕셔너리로 저장
            result_dict = {
                "단지명": apart_name,
                "단지 위도": latitude,
                "단지 경도": longitude,
                "지하철역": nearest_subway_station['name'],
                "거리(m)": round(float(nearest_subway_station['distance']), 2)
            }
            result_list.append(result_dict)
    else:
        # 원래 값 그대로 사용
        result_dict = {
            "단지명": apart_name,
            "단지 위도": latitude,
            "단지 경도": longitude,
            "지하철역": subway_station,  # 원래 값 그대로 사용
            "거리(m)": distance_subway # 거리는 0으로 대체
        }
        result_list.append(result_dict)

# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(result_list)

# 결과를 엑셀 파일로 저장
#result_df.to_csv('거리계산(중복제거)/DB_변수추가/분석용데이터_전처리완/연립다세대_매매_이상치처리.csv', index=False) 
result_df


C:\Users\User\AppData\Local\Temp\ipykernel_8616\197566785.py:72: DtypeWarning: Columns (6,16) have mixed types. Specify dtype option on import or set low_memory=False.
  apart = pd.read_csv(excel_file_path, encoding='euc-kr')[2515:2520]
100%|██████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 2498.69it/s]


,단지명,단지 위도,단지 경도,지하철역,거리(m)
0,(96-33),37.591120,127.091099,면목역 7호선,414.57
1,(96-33),37.591120,127.091099,면목역 7호선,414.57
2,대혁빌라,37.515118,126.839648,신정네거리역 2호선,1298.54
3,대혁주택나동,37.520911,126.833246,까치산역 2호선,1679.86
4,대혁주택나동,37.520911,126.833246,까치산역 2호선,1679.86


In [1]:
# import pandas as pd

# # CSV 파일에서 데이터 읽기 (인코딩 변경)
# csv_file_path = '데이터변수 정리/2024년1~4월1일기준_서울시버스정류소위치정보.csv'
# bus = pd.read_csv(csv_file_path, delimiter=',', encoding='euc-kr')  # EUC-KR 인코딩 시도

# # Excel 파일에서 데이터 읽기
# excel_file_path = '좌표변환_전체아파트_19-23.xlsx'
# apart = pd.read_excel(excel_file_path)
# bus, apart #데이터 확인

In [2]:
#버스 - 파일 이용 

In [20]:
import pandas as pd
from geopy.distance import geodesic
from tqdm import tqdm

# 버스 정류소 데이터 로드
csv_file_path = '데이터변수 정리/2024년1~4월1일기준_서울시버스정류소위치정보.csv'
bus_stops = pd.read_csv(csv_file_path, delimiter=',', encoding='euc-kr')  # EUC-KR 인코딩 시도
#bus_stops.rename(columns = {'STTN_NM':'bus_stop', 'CRDNT_X': 'longitude' ,'CRDNT_Y':'latitude'}, inplace=True)

apart = pd.read_csv('거리계산(중복제거)/DB_변수추가/연립다세대_전월세.csv', encoding='euc-kr')[22557:22560]


# 도로상의 거리를 구하는 함수 정의
def find_road_distance(apartment_latitude, apartment_longitude, stop_latitude, stop_longitude):
    # 거리 계산 (간단히 유클리디안 거리를 사용)
    distance = geodesic((apartment_latitude, apartment_longitude), (stop_latitude, stop_longitude)).meters
    return distance

# 각 아파트 단지에 대해 가장 가까운 정류소를 찾는 함수 정의
def find_nearest_bus_stop(apartment_latitude, apartment_longitude):
    closest_stop = None
    min_road_distance = float('inf')
    
    for idx, row in bus_stops.iterrows():
        stop_name = row['STTN_NM']  # 버스 정류소명 컬럼
        stop_latitude = row['CRDNT_Y']  # 정류소 위도
        stop_longitude = row['CRDNT_X']  # 정류소 경도
        
        # 도로상 거리 계산
        road_distance = find_road_distance(apartment_latitude, apartment_longitude, stop_latitude, stop_longitude)
        
        # 최소 도로상 거리인 경우 해당 정류소 정보 업데이트
        if road_distance < min_road_distance:
            closest_stop = stop_name
            min_road_distance = road_distance
    
    return closest_stop, min_road_distance

# 각 아파트 단지에 대해 가장 가까운 정류소 찾기
results = []

for idx, apt in tqdm(apart.iterrows(), total=apart.shape[0]):
    # NA 값이 포함된 행인지 확인
    if apt.isna().any():
        continue  # NA 값이 포함된 행이면 건너뛰기
    
    apt_name = apt['단지명']
    apt_latitude = apt['단지 위도']
    apt_longitude = apt['단지 경도']
    
    # 가장 가까운 정류소 찾기
    nearest_stop, road_distance = find_nearest_bus_stop(apt_latitude, apt_longitude)
    
    # 해당 정류소의 좌표 가져오기
    nearest_stop_row = bus_stops[bus_stops['STTN_NM'] == nearest_stop]
    nearest_stop_latitude = nearest_stop_row.iloc[0]['CRDNT_Y']
    nearest_stop_longitude = nearest_stop_row.iloc[0]['CRDNT_X']
    
    # 결과 저장
    results.append({
        '단지명': apt_name,
        '단지 위도': apt_latitude,
        '단지 경도': apt_longitude,
        '가장 가까운 정류소': nearest_stop,
        '최단거리(m)': round(float(road_distance),2)
    })

# 결과를 데이터프레임으로 변환하여 출력
results_df = pd.DataFrame(results)
results_df
#result_df.to_excel('카운트된_버스_정보.xlsx', index=False)

C:\Users\User\AppData\Local\Temp\ipykernel_456\2938958486.py:10: DtypeWarning: Columns (8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  apart = pd.read_csv('거리계산(중복제거)/DB_변수추가/연립다세대_전월세.csv', encoding='euc-kr')[22557:22560]
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:57<00:00, 19.01s/it]


,단지명,단지 위도,단지 경도,가장 가까운 정류소,최단거리(m)
0,(1-23),37.561698,127.039513,왕십리민자역사,145.5
1,(1-23),37.561698,127.039513,왕십리민자역사,145.5
2,(1-23),37.561698,127.039513,왕십리민자역사,145.5


In [3]:
#지하철 - 카카오API

In [64]:
import pandas as pd
import math
import requests
from tqdm import tqdm

# 카카오맵 API 키
API_KEY = "b3d07b3bc9d36308dfc4b023582e291d"

# 검색 반경 (미터)
search_radius = 5000

def calculate_distance(lat1, lon1, lat2, lon2):
    # 지구 반지름 (단위: km)
    R = 6371.0

    # 라디안으로 변환
    lat1_rad = math.radians(float(lat1))
    lon1_rad = math.radians(float(lon1))
    lat2_rad = math.radians(float(lat2))
    lon2_rad = math.radians(float(lon2))

    # 위도 및 경도 간의 차이 계산
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # Haversine 공식을 사용하여 거리 계산
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c * 1000 #m단위로 변경

    return distance

def get_nearest_subway_station(latitude, longitude, radius, api_key):
    url = "https://dapi.kakao.com/v2/local/search/category.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {
        "category_group_code": "SW8",  # SW8는 지하철역 카테고리 코드
        "y": latitude,
        "x": longitude,
        "radius": radius,
        "sort": "distance",
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if "meta" in data and data["meta"]["total_count"] > 0:
        nearest_station = data["documents"][0]

        # 선택된 지하철역 정보 추출
        station_name = nearest_station["place_name"]
        station_latitude = round(float(nearest_station["y"]), 6)
        station_longitude = round(float(nearest_station["x"]), 6)
        station_distance = calculate_distance(latitude, longitude, station_latitude, station_longitude)

        return {
            "name": station_name,
            "latitude": station_latitude,
            "longitude": station_longitude,
            "distance": station_distance
        }
    else:
        return None

# 엑셀 파일 경로
excel_file_path = '거리계산(중복제거)/DB_변수추가/분석용데이터_전처리완/연립다세대_매매_이상치처리 - 복사본.csv'

# 엑셀 파일에서 데이터 프레임으로 읽어오기
apart = pd.read_csv(excel_file_path, encoding = 'euc-kr')
apart = apart[apart['지하철역']=='0']
apart = apart[10051:]

# 결과를 저장할 리스트 초기화
result_list = []

# 각 단지명에 대해 가장 가까운 지하철역 정보 찾기
for index, row in tqdm(apart.iterrows(), total=apart.shape[0]):
    apart_name = row['단지명']
    latitude = row['단지 위도']
    longitude = row['단지 경도']
    if row.isna().any():
        continue  # NA 값이 포함된 행이면 건너뛰기

    # 주어진 위치 주변의 가장 가까운 지하철역 찾기
    nearest_subway_station = get_nearest_subway_station(latitude, longitude, search_radius, API_KEY)
    if nearest_subway_station:
        # 결과를 딕셔너리로 저장
        result_dict = {
            "단지명": apart_name,
            "단지 위도": latitude,
            "단지 경도": longitude,
            "지하철역": nearest_subway_station['name'],
            "거리(m)": round(float(nearest_subway_station['distance']),2)
        }
        result_list.append(result_dict)
    result_df2 = pd.DataFrame(result_list)
    result_df2.to_excel('거리계산(중복제거)/DB_변수추가/분석용데이터_전처리완/지하철수정4.xlsx', index=False) 

result_df2

100%|██████████████████████████████████████████████████████████████████████████| 11464/11464 [4:01:20<00:00,  1.26s/it]


,단지명,단지 위도,단지 경도,지하철역,거리(m)
0,복지빌라,37.606243,126.909085,구산역 6호선,909.97
1,복지빌라,37.587338,127.099566,면목역 7호선,1073.79
2,복지빌라,37.587215,127.099742,면목역 7호선,1091.15
3,복지빌라,37.587215,127.099742,면목역 7호선,1091.15
4,복지빌라,37.587101,127.099694,면목역 7호선,1088.99
...,...,...,...,...,...
11458,신일그린빌(B),37.639222,127.016316,가오리역 우이신설선,261.26
11459,신일라이프(A동),37.651307,127.025696,쌍문역 4호선,846.59
11460,신일라이프(A동),37.651307,127.025696,쌍문역 4호선,846.59
11461,신일맨션,37.599860,126.927294,녹번역 3호선,759.17


In [ ]:
#초중고 - 카카오API이용

In [17]:
import pandas as pd
import math
import requests
from tqdm import tqdm

# 카카오맵 API 키
API_KEY = "7d6ff50838116e9fdabd2726767fad40"

# 검색 반경 (미터)
search_radius = 5000

def calculate_distance(lat1, lon1, lat2, lon2):
    # 지구 반지름 (단위: km)
    R = 6371.0

    # 라디안으로 변환
    lat1_rad = math.radians(float(lat1))
    lon1_rad = math.radians(float(lon1))
    lat2_rad = math.radians(float(lat2))
    lon2_rad = math.radians(float(lon2))

    # 위도 및 경도 간의 차이 계산
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # Haversine 공식을 사용하여 거리 계산
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c * 1000

    return distance

def get_nearest_school(latitude, longitude, radius, school_type, api_key):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {
        "query": school_type,
        "y": latitude,
        "x": longitude,
        "radius": radius,
        "sort": "distance",
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if "meta" in data and data["meta"]["total_count"] > 0:
        # 검색 결과에서 특정 키워드로 끝나는 항목만 필터링
        filtered_results = [item for item in data["documents"] if item["category_name"].endswith(school_type)]

        if filtered_results:
            nearest_school = filtered_results[0]

            # 선택된 학교 정보 추출
            school_name = nearest_school["place_name"]
            school_latitude = nearest_school["y"]
            school_longitude = nearest_school["x"]
            school_distance = calculate_distance(latitude, longitude, school_latitude, school_longitude)

            return {
                "name": school_name,
                "latitude": school_latitude,
                "longitude": school_longitude,
                "distance": school_distance
            }
        else:
            return None
    else:
        return None

# Excel 파일 경로 설정
df = pd.read_csv('거리계산(중복제거)/DB_변수추가/연립다세대_전월세.csv', encoding='euc-kr')[22557:22560]

# latitude와 longitude 열을 숫자형(실수)으로 변환
df['단지 위도'] = pd.to_numeric(df['단지 위도'])
df['단지 경도'] = pd.to_numeric(df['단지 경도'])

# 검색할 키워드 리스트
school_types = ["초등학교", "중학교", "고등학교"]

# 데이터프레임의 각 행에 대해 주변 학교 검색
school_data = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    row_data = {"단지명": row['단지명'], "단지 위도": row['단지 위도'], "단지 경도": row['단지 경도']}
    if row.isna().any():
        continue  # NA 값이 포함된 행이면 건너뛰기

    for school_type in school_types:
        nearest_school = get_nearest_school(row['단지 위도'], row['단지 경도'], search_radius, school_type, API_KEY)
        if nearest_school:
            row_data[f"{school_type}"] = nearest_school['name']
            row_data[f"{school_type} 거리(m)"] = round(float(nearest_school['distance']),2)
        else:
            row_data[f"{school_type}"] = None
            row_data[f"{school_type} 거리(m)"] = None

    school_data.append(row_data)

# 결과 데이터프레임 생성
result_df3 = pd.DataFrame(school_data)
print(result_df3)

# 결과 데이터프레임을 엑셀 파일로 저장
#result_df3.to_excel('학교수정.xlsx', index=False)

C:\Users\User\AppData\Local\Temp\ipykernel_456\1370402101.py:72: DtypeWarning: Columns (8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('거리계산(중복제거)/DB_변수추가/연립다세대_전월세.csv', encoding='euc-kr')[22557:22560]
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.11it/s]

      단지명      단지 위도       단지 경도      초등학교  초등학교 거리(m)    중학교  중학교 거리(m)  \
0  (1-23)  37.561698  127.039513  서울행당초등학교      716.06  동마중학교     117.68   
1  (1-23)  37.561698  127.039513  서울행당초등학교      716.06  동마중학교     117.68   
2  (1-23)  37.561698  127.039513  서울행당초등학교      716.06  동마중학교     117.68   

               고등학교  고등학교 거리(m)  
0  한양대학교사범대학 부속고등학교      249.95  
1  한양대학교사범대학 부속고등학교      249.95  
2  한양대학교사범대학 부속고등학교      249.95  


In [ ]:
#병원 개수 - 서울시 병의원 위치 정보.csv 사용

In [78]:
import pandas as pd
import math
from tqdm import tqdm

# 검색 반경 (미터)
search_radius = 1300

def calculate_distance(lat1, lon1, lat2, lon2):
    # 지구 반지름 (단위: km)
    R = 6371.0

    # 라디안으로 변환
    lat1_rad = math.radians(float(lat1))
    lon1_rad = math.radians(float(lon1))
    lat2_rad = math.radians(float(lat2))
    lon2_rad = math.radians(float(lon2))

    # 위도 및 경도 간의 차이 계산
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # Haversine 공식을 사용하여 거리 계산
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c * 1000 # m단위로 변환

    return distance

def count_hospitals_nearby(apart_name, latitude, longitude, hospitals_data, max_distance):
    count = 0
    for index, hospital in hospitals_data.iterrows():
        hospital_latitude = hospital['병원위도']
        hospital_longitude = hospital['병원경도']
        distance = calculate_distance(latitude, longitude, hospital_latitude, hospital_longitude)
        if distance <= max_distance:
            count += 1
    return count

# 엑셀 파일 경로
csv_file_path = "서울시 병의원 위치 정보.csv"

# CSV 파일에서 병원 데이터 불러오기
hospitals_data = pd.read_csv(csv_file_path, encoding='cp949')

# 엑셀 파일에서 데이터 프레임으로 읽어오기
apart = pd.read_excel("거리계산(중복제거)/좌표변환_전체아파트_19-23(중복제거).xlsx")[1920:1925]

# 결과를 저장할 리스트 초기화
result_list = []

# 각 단지명에 대해 1300m 이내에 있는 병원의 개수 카운트
for index, row in tqdm(apart.iterrows(), total=apart.shape[0]):
    apart_name = row['단지명']
    apart_latitude = row['단지 위도']
    apart_longitude = row['단지 경도']
    if row.isna().any():
        continue  # NA 값이 포함된 행이면 건너뛰기

    # 1300m 이내에 있는 병원의 개수 카운트
    hospital_count = count_hospitals_nearby(apart_name, apart_latitude, apart_longitude, hospitals_data, 1300)
    
    # 결과를 딕셔너리로 저장
    result_dict = {
        "단지명": apart_name,
        "단지 위도": apart_latitude,
        "단지 경도": apart_longitude,
        "병원(개)": hospital_count
    }
    result_list.append(result_dict)

# 결과 리스트를 데이터프레임으로 변환
result_df4 = pd.DataFrame(result_list)

# 결과 데이터프레임 출력
result_df4
# 결과 데이터프레임을 엑셀 파일로 저장
#result_df4.to_excel('카운트된_병원_정보.xlsx', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:07<00:00,  1.58s/it]


,단지명,단지 위도,단지 경도,병원(개)
0,대명사이버아파트,37.543452,127.140173,400
1,대명아파트,37.620176,127.081296,198
2,대명에버빌아파트,37.542139,127.138229,487
3,대명이룸아파트(565-0),37.564870,127.062361,220
4,대명하이빌(103동),37.497338,126.831469,77


In [ ]:
#경찰서(지구대, 파출소 포함), 공공도서관, 소방서 - 카카오API 이용

In [1]:
import pandas as pd
import requests
from geopy.distance import geodesic
from tqdm import tqdm

# Kakao REST API 키
REST_API_KEY = 'f3da4534e20ed82ac1571f3e8eba33c0'

# 검색 키워드들 설정
search_keywords = ['공공도서관', '경찰서', '소방서', '대형마트']

# 검색 반경 (1.3km)
search_radius = 1300

def get_nearby_places(location, keyword, radius, api_key):
    latitude, longitude = location
    # Kakao Local API 요청 URL 설정
    url = f"https://dapi.kakao.com/v2/local/search/keyword.json"
    
    # 요청 파라미터 설정
    params = {
        "y": latitude,
        "x": longitude,
        "radius": radius,
        "query": keyword
    }
    
    # Kakao Local API에 GET 요청 보내기
    response = requests.get(url, headers={"Authorization": f"KakaoAK {api_key}"}, params=params)
    
    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # 응답 JSON 파싱
        data = response.json()
        # 검색된 장소 가져오기
        places = data.get('documents', [])
        
        # 중복된 장소명 필터링을 위한 집합
        visited_places = set()
        unique_places = []
        
        for place in places:
            place_name = place.get('place_name', 'Unknown')
            place_latitude = place.get('y', 'Unknown')
            place_longitude = place.get('x', 'Unknown')
            
            # Calculate distance using geodesic from geopy
            place_distance = geodesic((latitude, longitude), (place_latitude, place_longitude)).meters
            
            # 장소명에서 공백을 기준으로 첫 번째 단어 추출
            first_word = place_name.split()[0]
            
            # 중복된 첫 단어인 경우 이미 방문한 장소로 처리
            if first_word not in visited_places:
                visited_places.add(first_word)
                unique_places.append({
                    'place_name': place_name,
                    'latitude': place_latitude,
                    'longitude': place_longitude,
                    'distance': place_distance
                })
        
        return unique_places
    
    else:
        print(f"Kakao Local API에서 {keyword} 검색을 실패했습니다.")
        return []

# 엑셀 좌표 파일 경로
excel_file_path = '좌표변환_전체아파트_19-23.xlsx'

# 엑셀 파일에서 좌표 데이터 프레임으로 읽어오기 (전체 행)
apart = pd.read_excel(excel_file_path)[:9000]

# 결과를 저장할 딕셔너리 초기화
result_data = {
    '단지명': [],
    '단지 위도': [],  
    '단지 경도': [], 
    '공공도서관(개)': [],
    '경찰서(개)': [],
    '소방서(개)': [],
    '대형마트(개)': []
}

# 각 단지명별로 검색 키워드의 가까운 장소 정보 가져오기
for index, row in tqdm(apart.iterrows(), total=apart.shape[0]):
    apart_name = row['단지명']
    apart_location = (row['latitude'], row['longitude'])
    if row.isna().any():
        continue  # NA 값이 포함된 행이면 건너뛰기
    
    # 결과 딕셔너리에 단지명과 해당 위치 정보 추가
    result_data['단지명'].append(apart_name)
    result_data['단지 위도'].append(row['latitude'])  # 단지명의 위도 추가
    result_data['단지 경도'].append(row['longitude'])  # 단지명의 경도 추가
    
    # 각 키워드별로 가까운 장소 정보 가져오기
    for keyword in search_keywords:
        places = get_nearby_places(apart_location, keyword, search_radius, REST_API_KEY)
        place_count = len(places)
        result_data[f'{keyword}(개)'].append(place_count)

# 결과 데이터프레임 생성
result_df5 = pd.DataFrame(result_data)

# 결과 데이터프레임 출력
#result_df5

# 결과 데이터프레임을 엑셀 파일로 저장
result_df5.to_excel('카운트된_공공도서관_정보_아파트1.xlsx', index=False)

100%|████████████████████████████████████████████████████████████████████████████| 9000/9000 [1:02:05<00:00,  2.42it/s]


In [ ]:
#편의점, 약국 개수

In [28]:
import pandas as pd
import requests
from geopy.distance import geodesic
from tqdm import tqdm

# Kakao REST API 키
REST_API_KEY = 'b3d07b3bc9d36308dfc4b023582e291d'

# 검색 키워드들 설정 (편의점과 약국)
search_keywords = ['편의점', '약국']

# 검색 반경 (300m)
search_radius = 300

def get_nearby_places(location, keyword, radius, api_key):
    latitude, longitude = location
    # Kakao Local API 요청 URL 설정
    url = f"https://dapi.kakao.com/v2/local/search/keyword.json"
    
    # 요청 파라미터 설정
    params = {
        "y": latitude,
        "x": longitude,
        "radius": radius,
        "query": keyword
    }
    
    # Kakao Local API에 GET 요청 보내기
    response = requests.get(url, headers={"Authorization": f"KakaoAK {api_key}"}, params=params)
    
    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # 응답 JSON 파싱
        data = response.json()
        # 검색된 장소 가져오기
        places = data.get('documents', [])
        return places
    else:
        print(f"Kakao Local API에서 {keyword} 검색을 실패했습니다.")
        return []

# 엑셀 파일 경로
excel_file_path = '거리계산(중복제거)/좌표변환_연립다세대_19-23(중복제거).xlsx'

# 엑셀 파일에서 데이터 프레임으로 읽어오기 (상위 10개 행)
apart = pd.read_excel(excel_file_path)[9775:9780]

# 결과를 저장할 딕셔너리 초기화
result_data = {
    '단지명': [],
    '단지 위도': [],  # 단지명의 위도
    '단지 경도': [],  # 단지명의 경도
    '편의점(개)': [],
    '약국(개)': []
}

# 각 단지명별로 검색 키워드의 가까운 장소 정보 가져오기
for index, row in tqdm(apart.iterrows(), total=apart.shape[0]):
    apart_name = row['단지명']
    apart_location = (row['latitude'], row['longitude'])
    if row.isna().any():
        continue  # NA 값이 포함된 행이면 건너뛰기

    # 결과 딕셔너리에 단지명과 해당 위치 정보 추가
    result_data['단지명'].append(apart_name)
    result_data['단지 위도'].append(row['latitude'])  # 단지명의 위도 추가
    result_data['단지 경도'].append(row['longitude'])  # 단지명의 경도 추가
    
    # 각 키워드별로 가까운 장소 정보 가져오기 (편의점과 약국만)
    for keyword in search_keywords:
        places = get_nearby_places(apart_location, keyword, search_radius, REST_API_KEY)
        place_count = len(places)
        result_data[f'{keyword}(개)'].append(place_count)  # 시설 옆에 '(개)' 추가

# 결과 데이터프레임 생성
result_df6 = pd.DataFrame(result_data)
# 결과 데이터프레임 출력
result_df6
# 결과 데이터프레임을 엑셀 파일로 저장
#result_df6.to_excel('카운트된_편의점_약국_정보.xlsx', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  5.03it/s]


,단지명,단지 위도,단지 경도,편의점(개),약국(개)
0,동신하이츠맨션(365-17),37.485410,126.853753,6,5
1,힐하우스(941-12),37.485416,126.990637,4,0
2,(321-4),37.485428,126.992880,8,2
3,한라빌라트,37.485428,127.131018,6,1
4,스위트서초,37.485429,127.014572,15,12


In [ ]:
#좌표변환 파일 중복제거

In [39]:
import pandas as pd

# 데이터 불러오기
data = pd.read_excel("좌표변환_전체연립다세대_19-23(수정).xlsx")

# 소수점 아래 6자리까지만 유지
data['latitude'] = data['latitude'].round(6)
data['longitude'] = data['longitude'].round(6)

# 중복된 행 제거
df_no_duplicates = data.drop_duplicates()

# 중복 제거된 데이터프레임을 엑셀 파일로 저장
df_no_duplicates.to_excel("중복제거2.xlsx", index=False)

In [ ]:
#중복제거 

In [13]:
import pandas as pd

# 데이터프레임 생성 예시
data = pd.read_excel("거리계산(중복제거)/거리계산_최종/학교_아파트(중복제거).xlsx")

df = pd.DataFrame(data)

df_no_duplicates = df.drop_duplicates()

df_no_duplicates.to_excel('거리계산(중복제거)/거리계산_최종/학교_아파트_중복제거(최종).xlsx', index = False)

In [ ]:
#파일 병합

In [70]:
import pandas as pd

# 첫 번째 Excel 파일 불러오기
df1 = pd.read_excel("거리계산(중복제거)/거리계산_최종/병원_연립다세대(중복제거).xlsx")

# 두 번째 Excel 파일 불러오기
df2 = pd.read_excel("거리계산(중복제거)/거리계산_최종/버스_지하철_공공도서관_편의점_학교_연립다세대.xlsx")

# 필요한 열만 선택
df1_selected = df1[['단지명', '단지 위도',  '단지 경도']]
df2_selected = df2[['단지명', '단지 위도', '단지 경도']]

# df1과 df2의 선택된 열을 나란히 합치기
merged_df = pd.concat([df1_selected, df2_selected], axis=1)


merged_df.to_excel('거리계산(중복제거)/거리계산_최종/좌표변환수정.xlsx',index=False)


In [ ]:
#파일 비교 -> 없는 행 찾기

In [4]:
import pandas as pd

# 첫 번째 Excel 파일 불러오기
df1 = pd.read_csv("거리계산(중복제거)/실거래가 연도별 병합/아파트_매매_좌표추가.csv", encoding='euc-kr')

# 두 번째 Excel 파일 불러오기
df2 = pd.read_excel("거리계산(중복제거)/좌표변환_전체아파트_19-23(중복제거).xlsx")

# 필요한 열만 선택
df1_selected = df1[['단지명', '단지 위도', '단지 경도']]
df2_selected = df2[['단지명', '단지 위도', '단지 경도']]

# 소수점 6자리까지 반올림
df1_selected_rounded = df1_selected.round(6)
df2_selected_rounded = df2_selected.round(6)

# df1에는 있지만 df2에는 없는 행 찾기
merged_df = pd.merge(df1_selected_rounded, df2_selected_rounded, on=['단지명', '단지 위도', '단지 경도'], how='left', indicator=True)
not_in_df2 = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

# 결과 출력
print("df2에 없는 행:")
print(not_in_df2)


df2에 없는 행:
                단지명      단지 위도       단지 경도
19             에스케이  37.568568  127.086382
34             삼보드림  37.555973  127.092352
37          포레스트힐시티  37.571278  127.026522
50              두산5  37.577959  127.081092
56            마젤란21  37.587152  127.084138
...             ...        ...         ...
8862          예림아너스  37.562211  126.916610
8869     세운푸르지오헤리시티  37.564566  126.996645
8871           라인캐슬  37.609362  127.053617
8873  힐스테이트세운센트럴2단지  37.568100  126.994414
8874  힐스테이트세운센트럴1단지  37.568096  126.993126

[639 rows x 3 columns]


In [ ]:
#거리계산 출력 -> 리스트로 확인

In [ ]:
#공공도서관, 소방서, 경찰서, 병원 카운트된 장소 결과
'''
import pandas as pd
import requests
from geopy.distance import geodesic

# Kakao REST API 키
REST_API_KEY = 'f3da4534e20ed82ac1571f3e8eba33c0'

# 검색 키워드들 설정
search_keywords = ['공공도서관', '소방서', '경찰서', '병원']

# 검색 반경 (2km)
search_radius = 1300

def get_nearby_places(location, keyword, radius, api_key):
    latitude, longitude = location
    # Kakao Local API 요청 URL 설정
    url = f"https://dapi.kakao.com/v2/local/search/keyword.json"
    
    # 요청 파라미터 설정
    params = {
        "y": latitude,
        "x": longitude,
        "radius": radius,
        "query": keyword
    }
    
    # Kakao Local API에 GET 요청 보내기
    response = requests.get(url, headers={"Authorization": f"KakaoAK {api_key}"}, params=params)
    
    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # 응답 JSON 파싱
        data = response.json()
        # 검색된 장소 가져오기
        places = data.get('documents', [])
        
        # 중복된 장소명 필터링을 위한 집합
        visited_places = set()
        unique_places = []
        
        for place in places:
            place_name = place.get('place_name', 'Unknown')
            place_latitude = place.get('y', 'Unknown')
            place_longitude = place.get('x', 'Unknown')
            
            # Calculate distance using geodesic from geopy
            place_distance = geodesic((latitude, longitude), (place_latitude, place_longitude)).meters
            
            # 장소명에서 공백을 기준으로 첫 번째 단어 추출
            first_word = place_name.split()[0]
            
            # 중복된 첫 단어인 경우 이미 방문한 장소로 처리
            if first_word not in visited_places:
                visited_places.add(first_word)
                unique_places.append({
                    'place_name': place_name,
                    'latitude': place_latitude,
                    'longitude': place_longitude,
                    'distance': place_distance
                })
        
        return unique_places
    
    else:
        print(f"Kakao Local API에서 {keyword} 검색을 실패했습니다.")
        return []

# 엑셀 파일 경로
excel_file_path = '좌표변환_아파트(매매)__2019.xlsx'

# 엑셀 파일에서 데이터 프레임으로 읽어오기 (상위 10개 행)
apart = pd.read_excel(excel_file_path)[:1]

# 결과를 저장할 딕셔너리 초기화
result_data = {
    '단지명': [],
    '공공도서관': [],
    '소방서': [],
    '경찰서': [],
    '병원': []
}

# 각 단지명별로 검색 키워드의 가까운 장소 정보 가져오기
for index, row in apart.iterrows():
    apart_name = row['단지명']
    apart_location = (row['latitude'], row['longitude'])
    
    result_data['단지명'].append(apart_name)
    
    # 각 키워드별로 가까운 장소 정보 가져오기
    for keyword in search_keywords:
        places = get_nearby_places(apart_location, keyword, search_radius, REST_API_KEY)
        place_count = len(places)
        result_data[keyword].append(place_count)
        
        # 검색된 장소 정보 출력
        if places:
            print(f"**{apart_name} 주변 {keyword}**")
            for place in places:
                place_name = place['place_name']
                place_latitude = place['latitude']
                place_longitude = place['longitude']
                place_distance = place['distance']
                
                print(f"장소명: {place_name},거리: {place_distance}m")
            print()

# 결과 데이터프레임 생성
result_df = pd.DataFrame(result_data)

# 결과 데이터프레임 출력
print(result_df)
'''

In [ ]:
#편의점, 약국 카운트된 장소명, 좌표 확인
'''
import pandas as pd
import requests
from geopy.distance import geodesic

# Kakao REST API 키
REST_API_KEY = 'f3da4534e20ed82ac1571f3e8eba33c0'

# 검색 키워드들 설정 (편의점과 약국 추가)
search_keywords = ['편의점', '약국']

# 검색 반경 (500m)
search_radius = 350

def get_nearby_places(location, keyword, radius, api_key):
    latitude, longitude = location
    # Kakao Local API 요청 URL 설정
    url = f"https://dapi.kakao.com/v2/local/search/keyword.json"
    
    # 요청 파라미터 설정
    params = {
        "y": latitude,
        "x": longitude,
        "radius": radius,
        "query": keyword
    }
    
    # Kakao Local API에 GET 요청 보내기
    response = requests.get(url, headers={"Authorization": f"KakaoAK {api_key}"}, params=params)
    
    # 요청이 성공했는지 확인
    if response.status_code == 200:
        # 응답 JSON 파싱
        data = response.json()
        # 검색된 장소 가져오기
        places = data.get('documents', [])
        return places
    else:
        print(f"Kakao Local API에서 {keyword} 검색을 실패했습니다.")
        return []

# 엑셀 파일 경로
excel_file_path = '좌표변환_아파트(매매)__2019.xlsx'

# 엑셀 파일에서 데이터 프레임으로 읽어오기 (상위 10개 행)
apart = pd.read_excel(excel_file_path)[:4]

# 결과를 저장할 딕셔너리 초기화
result_data = {
    '단지명': [],
    '편의점': [],
    '약국': []
}

# 각 단지명별로 검색 키워드의 가까운 장소 정보 가져오기
for index, row in apart.iterrows():
    apart_name = row['단지명']
    apart_location = (row['latitude'], row['longitude'])
    
    result_data['단지명'].append(apart_name)
    
    # 각 키워드별로 가까운 장소 정보 가져오기 (편의점과 약국만)
    for keyword in search_keywords:
        places = get_nearby_places(apart_location, keyword, search_radius, REST_API_KEY)
        place_count = len(places)
        result_data[keyword].append(place_count)
        
        # 검색된 장소 정보 출력
        if places:
            print(f"**{apart_name} 주변 {keyword}**")
            for place in places:
                place_name = place.get('place_name', 'Unknown')
                place_latitude = place.get('y', 'Unknown')
                place_longitude = place.get('x', 'Unknown')
                
                # Calculate distance using geodesic from geopy
                place_distance = geodesic(apart_location, (place_latitude, place_longitude)).meters
                
                print(f"장소명: {place_name}, 위도: {place_latitude}, 경도: {place_longitude}, 거리: {place_distance}m")
            print()

# 결과 데이터프레임 생성
result_df = pd.DataFrame(result_data)

# 결과 데이터프레임 출력
print(result_df)
'''